# Pushdowm Automata

In [1]:
import warnings

class pushdown_automata:
    
    def __init__(self, q = None, sigma = None, sigma_stack = None, delta = None, q0 = None, empty_symbol = "lambda", f = None):
        """
        Class initializar
        
        self.q receives all the states that conform the pushdown automata
        
        self.sigma are all the variables defined in the pushdown automata
        
        self.sigma_stack all the variables defined for the stack
        
        self.delta all are the operation defined, i.e- is a dictionary contating a tuple as key
        where the left values of the tuple is the current state and the right value the variable pass and the value obtain by
        accessing to with the key is to which state goes.
        
        self.q0 is the initial state
        
        self.empty_symbol is the symbol in which is represent the empty value, for instance, it is lambda bout could be epsilon
        or another symbol, but it has to be defined, it is not receive from txt.
        
        self.f are all the final states
        
        self.stack_operation 
        
        self.stack list saving the stack elements
        
        self.actual_state is the state which the dfa finish after proccessing a word, if no word proccess the actual state is q0
        and it is reset at start proccessing a new word
        """
        self.q = q
        self.sigma = sigma
        self.sigma_stack = sigma_stack
        self.delta = delta
        self.q0 = q0
        self.empty_symbol = empty_symbol
        self.f = f
        
        
        self.stack = list()
        self.stack_operation = None
        self.actual_state = q0
        
        
        self.trampa = False
        
        try:
            if self.is_valid():
                pass

            else:
                warnings.warn("Not valid automaton")

        except:
            warnings.warn("Not valid automaton")
            
        
        
        
    def load_from_file(self, path, delta_separator = "-"):
        """
        Function to read a file that contains an automaton
        
        path: path file
        """
        
        #dictionary to save the parts of the automaton
        variables = {"q": None,
             "sigma": None,
             "sigma_stack": None,
             "delta": None,
             "q0": None,
             "f": None    
        }
        
        stack_operations = dict()
        
        with open(path) as file:
            #parsing the document
            for line, variable in zip(file, variables):
                    line = line.replace("{", "")
                    line = line.replace("}", "")
                    line = line.replace("\n", "")
                    line = line.replace(" ", "")
                    value = list(line.split(","))
                    variables[variable] = value
                    
            #conditional to verify there is only one initial state        
            if len(variables["q0"]) > 1 or len(variables["q0"]) == 0:
                raise Exception('There are more than one initial state')
                
            variables["delta"] = [values.split(delta_separator) for values in variables["delta"]]
            
            for i in variables["delta"]:
                if (i[0],i[1],i[2]) in stack_operations:
                    stack_operations[(i[0],i[1],i[2])].append(i[3])
                else:
                    stack_operations[(i[0],i[1],i[2])] = [i[3]]
        
        
            new_delta = dict()
            
            for i in variables["delta"]:
                if (i[0],i[1],i[2]) in new_delta:
                    new_delta[(i[0],i[1],i[2])].append(i[4])
                    
                else:
                    new_delta[(i[0],i[1],i[2])] = [i[4]]
        
        
            
            
            #getting q0
            variables["q0"] = variables["q0"][0]
            
            #assigning the attributos of the class to the values obtained from the file
            self.q = variables["q"]
            self.sigma = variables["sigma"]
            self.delta = new_delta
            self.q0 = variables["q0"] 
            self.f = variables["f"]
            self.sigma_stack = variables['sigma_stack']
            self.stack_operation = stack_operations
            self.actual_state = variables["q0"]
            
            if self.is_valid():
                pass
            
            else:
                raise ErrorConfiguration("Not valid automaton")
            
            
            
    def is_valid(self):
        
        """
        Funtion to validate whether the pushdown is valid or not
        
        Uses the attributes of the class
        """
        
        
        #get the values of the stack operations
        all_to_go_states = [value for value in self.delta.values()]
        
        all_to_go_states = [item for sublist in all_to_go_states for item in sublist ]
        
        #get the values of the deltas
        all_to_stack_to_push = [value for value in self.stack_operation.values()]
        
        all_to_stack_to_push = [item for sublist in all_to_stack_to_push for item in sublist if item != self.empty_symbol]
        
        #gets the left value of the tuple
        all_states_to_receive = set([state[0] for state in list(self.delta.keys())])
        
        #gets the right value of the tuple
        all_variables_to_receive = set([state[1] for state in list(self.delta.keys())])
        
        #get the value of the stack element to consume
        all_variables_to_consume = set([state[2] for state in list(self.delta.keys()) if state[2] != self.empty_symbol])
        
        #conditional to check if the final states are subsets of q, which is the variable containing all the states
        if set(self.f).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("States of F are not a subset of states q")
            
            
        #conditional to check if the inital state is a subsets of q, which is the variable containing all the states    
        if self.q0 in set(self.q):
            pass
        
        else:
            raise TypeError("q0 is not a subset of states q")
            
        
        #checking if the left value of the tuple is subset of q and the right value is subset of sigma
        if set(all_states_to_receive).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("The state before a applying the transiction is not a subset of q")
            
        sigmas = set(self.sigma)
        sigmas.add(self.empty_symbol)
        if set(all_variables_to_receive).issubset(sigmas):
            pass
        
        else:
            print(all_variables_to_receive)
            print(self.sigma)
            raise TypeError("The variable which is apply to the transiction is not a subset of sigma")
        
        #checking if all the transictions of delta are subsets of q
        if set(all_to_go_states).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("The state resultant of a transiction is not a subset of states q")
            
         #checking if all the transictions of delta are subsets of q
        if set(all_variables_to_consume).issubset(set(self.sigma_stack)):
            pass
        
        else:
            raise TypeError("The stack element to consume does not belong to the variables for the stack")
            
        
        #checking if all the transictions of delta are subsets of q
        if set(all_to_stack_to_push).issubset(set(self.sigma_stack)):
            pass
        
        else:
            raise TypeError("The stack element to push does not belong to the variables for the stack")
            
        return True
            
            
    def empty_transitions(self, state, stack_element = None):
        """
        Function to find all reachable states from an initial state just passing empty symbols

        Input:
        Current State
        stack_element: is an stack element consumed in the stack in order to apply that empty transition, this stack element could be 
        an empty symbol as well.

        Output:
        return a tuple of 2 elements that each one is a list of value of a given key
        """
        #case where a stack element was not passed, so it
        if stack_element == None:
            stack_element = self.empty_symbol
        
        #check if the state belongs to the automata states
        if state in set(self.q):
            pass
        else:
            raise Exception("The given state does not belong to this automaton")

        
        if stack_element != self.empty_symbol:
            #check if the stack element belongs to the variables of the stack of the automata
            if stack_element in set(self.sigma_stack):
                pass
            else:
                raise Exception("The given stack element does not belong to this automaton")

        
        try:
            path = self.delta[(state, self.empty_symbol, stack_element)]
            element_to_stack = self.stack_operation[(state, self.empty_symbol, stack_element)]

            return path, element_to_stack
        
        except:
            raise ErrorConfiguration("That transiction does not exist")
    
    
    
    def process_symbol(self, state, symbol, stack_element):
        """
        Function to find all the states you can reach from one state processing a symbol and an stack element
        
        Input:
        state: current state
        
        symbol: variable belonging to sigma
        
        stack_element: element of the stack
        
        showing_backtracking: boolean to know whether to print the path or not
        
        Output:
        return a tuple of 2 elements that each one is a list of value of a given key
        """
    
        try:
            path = self.delta[(state, symbol, stack_element)]
            element_to_stack = self.stack_operation[(state, symbol, stack_element)]

            return path, element_to_stack

        except:
            raise ErrorConfiguration("That transiction does not exist")
        
        
    def _process_empty_string(self, state, paths = [], showing_backtrack = True):
        """
        Function to check if an empty string or '' is valid for the automaton
        
        if it is possible that '' is accepted by the automata, thus return a True and the path that it takes
        """
        paths.append(state)
        
        #reach a final state
        if state in self.f:
            self.trampa = True
            return 
        
        conditional = False
        
        stack_memory_primal = self.stack[:]
        
        #case when only consuming empty symbol
        if (state, self.empty_symbol, self.empty_symbol) in self.delta:
            conditional = True
            
            path, element_to_stack = self.empty_transitions(state, self.empty_symbol)

            for path_, element_ in zip(path, element_to_stack):
                paths.append("{}, {} --> {}".format(self.empty_symbol, self.empty_symbol, element_))
                paths.append(path_)
                if showing_backtrack:
                    print(paths)
                if element_ != self.empty_symbol:
                    self.stack.append(element_)


                stack_memory = self.stack[:]
                self._process_empty_string(path_, paths, showing_backtrack)
                if self.trampa:
                    return

                self.stack = stack_memory[:]

                paths.pop()
                paths.pop()
                
            self.stack = stack_memory_primal[:]
              
        #case consuming an element of the stack
        if (state, self.empty_symbol, self.stack[-1]):
            conditional = True
            
            path, element_to_stack = self.empty_transitions(state, self.stack[-1])
            
            memory = self.stack[-1]
            
            self.stack.pop(-1)

            for path_, element_ in zip(path, element_to_stack):
                paths.append("{}, {} --> {}".format(self.empty_symbol, memory, element_))
                paths.append(path_)
                if showing_backtrack:
                    print(paths)
                if element_ != self.empty_symbol:
                    self.stack.append(element_)


                stack_memory = self.stack[:]
                self._process_empty_string(path_, paths, showing_backtrack)
                if self.trampa:
                    return

                self.stack = stack_memory[:]

                paths.pop()
                paths.pop()
                
            stack_memory_primal = self.stack[:]
            
        if not(conditional):
            return
            
            
    def process_word(self, word, showing_backtrack = True):
        """
        Function to determine if a word belong to the language of the ndfa
        
        input:
        Word: word to be proccessed
        
        Output:
        return a tuple of 2 elements, the first one is a boolean to know if the word has been processed or not, and the second element is the
        path of the processed word if True.
        """ 

        #memory
        paths = []
        
        #reset the stack
        self.stack = []
        
        #case to check if an empty string is acceptable or not
        if word == '':
            self._process_empty_string(self.q0, paths, showing_backtrack)
                
            if self.trampa:
                return self.trampa, paths
            else:
                return False
        
        #transforming the word into a str
        word = str(word)
        
        #check the word given is valid
        self.valid_string(word)
        
         
        #reseting attributes
        self.trampa = False
        
          
        #adding to the backtracking
        paths.append(self.q0)
        #going into recursive
        self._process_word_recursive(word, self.q0 ,paths, showing_backtrack)
        
        
        if self.trampa:
            return self.trampa, paths
            
        else:
            return self.trampa
        
    def _process_word_recursive(self,word,current_state,paths = [], showing_backtrack = True):
        """
        Function to determine if a word belong to the language of the ndfa
        
        input:
        Word: word to be proccessed
        
        Output:
        return a bolean whether if the word is proccessed by the automaton or not
        """ 

        #check if the word is not empty
        try:
        
            letter = word[0]
        
        #case where is no more variables to consume
        except:
            #checking if the last state is in final states
            if current_state in list(self.f):
                if showing_backtrack:
                    print("---------------------------valid-----------------------\n\n")
                #saving that the word is valid   
                self.trampa = True
                #actul state to use in function str
                self.actual_state = current_state
                return 
            
            else:
                

                #case when the stack has elements left, so check if the automata can still move
                if len(self.stack) != 0:
                    
                    #reversing stack
                    for element in reversed(self.stack):
                        
                        #check if has reacheable states
                        if (current_state, self.empty_symbol, element) in self.delta:
                            path, element_to_stack = self.empty_transitions(current_state, element)

                            #eliminating element of the stack
                            self.stack.pop(-1)

                            current_state = path

                            #check if the given reacheable states are valid
                            for _state, _element in zip(current_state, element_to_stack):
                                if _state in self.f:
                                    paths.append("{}, {} --> {}".format(self.empty_symbol, element, _element))
                                    paths.append(_state)
                                    if showing_backtrack:
                                        print(paths)
                                    self.actual_state = current_state
                                    self.trampa = True
                                    return
                                
                        else:
                            if showing_backtrack:
                                print("---------------------------Unvalid---------------------------\n\n")
                            self.actual_state = current_state
                            return
                        
                
                #not valid 
                else:
                    if showing_backtrack:
                        print("---------------------------Unvalid---------------------------\n\n")
                        #the same as the part latter
                        
                    self.actual_state = current_state
                    return
            
            
        #check if the stack has no elements
        if len(self.stack) == 0:
            
            conditional = False
            #check if the combination of the current state consuming a letter of the word and an epsilon of the stack has reacheable states
            if (current_state, letter, self.empty_symbol) in self.delta:
                conditional = True
                
                path, element_to_stack = self.process_symbol(current_state, letter, self.empty_symbol)
                
                #check all the reacheable states with its element to push in the stack
                for path_, element_ in zip(path, element_to_stack):
                    
                    paths.append("{}, {} --> {}".format(letter, self.empty_symbol, element_))
                    paths.append(path_)
                    if showing_backtrack:
                        print(paths)
                    if element_ != self.empty_symbol:
                        self.stack.append(element_)

                    #memory to not lose the information of the stack in the recursive function
                    stack_memory = self.stack[:]
                    self._process_word_recursive(word[1:], path_, paths, showing_backtrack)
                    
                    if self.trampa:
                        return

                    self.stack = stack_memory[:]
                    paths.pop()
                    paths.pop()
                    
                    
            self.stack = []
            #option when is an empty transition
            if (current_state, self.empty_symbol, self.empty_symbol) in self.delta:
                
                conditional = True
                path, element_to_stack = self.empty_transitions(current_state)

                for path_, element_ in zip(path, element_to_stack):
                    paths.append("{}, {} --> {}".format(self.empty_symbol, self.empty_symbol, element_))
                    paths.append(path_)
                    if showing_backtrack:
                        print(paths)
                    if element_ != self.empty_symbol:
                        self.stack.append(element_)

                    stack_memory = self.stack[:]
                    self._process_word_recursive(word, path_, paths, showing_backtrack)
                    
                    if self.trampa:
                        return
                    self.stack = stack_memory[:]
                    paths.pop()
                    paths.pop()
                    

            #case does not have reacheable states
            if not(conditional):
                return

        else:
            #element for the case when there is not case there it gets a new state to reach
            conditional = False
            
            #memory before going to each possible if
            primal_stack_memory = self.stack[:]
            
            #case when consumes a leter and an stack element
            if (current_state, letter, self.stack[-1]) in self.delta:
                conditional = True
                path, element_to_stack = self.process_symbol(current_state, letter, self.stack[-1])

                memory = self.stack[-1]
                self.stack.pop(-1) 
                
                for path_, element_ in zip(path, element_to_stack):
                    paths.append("{}, {} --> {}".format(letter, memory, element_))
                    paths.append(path_)
                    if showing_backtrack:
                        print(paths)
                    
                    if element_ != self.empty_symbol:
                        self.stack.append(element_)
           
                    stack_memory = self.stack[:]
                    self._process_word_recursive(word[1:], path_, paths, showing_backtrack)
                    if self.trampa:
                        return
                    self.stack = stack_memory[:]
                    paths.pop()
                    paths.pop()
                    
                self.stack = primal_stack_memory[:]
                    
            #case when consumes an empty symbol and an stack elemeten
            if (current_state, self.empty_symbol, self.stack[-1]) in self.delta:

                conditional = True
                
                path, element_to_stack = self.empty_transitions(current_state, self.stack[-1])

                memory = self.stack[-1]
                self.stack.pop(-1)

                for path_, element_ in zip(path, element_to_stack):
                    paths.append("{}, {} --> {}".format(self.empty_symbol, memory, element_))
                    paths.append(path_)
                    if showing_backtrack:
                        print(paths)
                    if element_ != self.empty_symbol:
                        self.stack.append(element_)
                        
                        
                    stack_memory = self.stack[:]
                    self._process_word_recursive(word, path_, paths, showing_backtrack)
                    if self.trampa:
                        return
                    self.stack = stack_memory[:]
                    paths.pop()
                    paths.pop()
                    
                self.stack = primal_stack_memory[:]
                    
            #case when consumes a letter but not an stack element
            if (current_state, letter, self.empty_symbol) in self.delta:
            
                conditional = True
                
                path, element_to_stack = self.process_symbol(current_state, letter, self.empty_symbol)

                for path_, element_ in zip(path, element_to_stack):
                    paths.append("{}, {} --> {}".format(letter, self.empty_symbol, element_))
                    paths.append(path_)
                    if showing_backtrack:
                        print(paths)
                    if element_ != self.empty_symbol:
                        self.stack.append(element_)
                        
                        
                        
                    stack_memory = self.stack[:]
                    self._process_word_recursive(word[1:], path_, paths, showing_backtrack)
                    if self.trampa:
                        return
                    self.stack = stack_memory[:]
                    
                    paths.pop()
                    paths.pop()
                    
                self.stack = primal_stack_memory[:]
                    
            #case of empty transition
            if (current_state, self.empty_symbol, self.empty_symbol) in self.delta:
                
                conditional = True
                
                path, element_to_stack = self.empty_transitions(current_state, self.empty_symbol)

                for path_, element_ in zip(path, element_to_stack):
                    paths.append("{}, {} --> {}".format(self.empty_symbol, self.empty_symbol, element_))
                    paths.append(path_)
                    if showing_backtrack:
                        print(paths)
                    if element_ != self.empty_symbol:
                        self.stack.append(element_)
                        
                        
                    stack_memory = self.stack[:]
                    self._process_word_recursive(word, path_, paths, showing_backtrack)
                    if self.trampa:
                        return
                    
                    self.stack = stack_memory[:]
                    
                    paths.pop()
                    paths.pop()
                    
                self.stack = primal_stack_memory[:]

            #not reacheable states
            if not(conditional):
                return
            
            
   
        
            
    
    def valid_string(self, word):
        
        if set(word).issubset(set(self.sigma)):
            return
        else:
            raise Exception("Not valid word, contains variables that are not part of the language")
            
            
    def __str__(self):
        
        
        cadena = "Grammar: \n q: {} \n sigma: {}, sigma stack: {},delta: {}, q0: {}, f: {}, current_sate: {}, empty_symbol: {}".format(
                                                                                    self.q , 
                                                                                    self.sigma,
                                                                                    self.sigma_stack,
                                                                                    self.delta, 
                                                                                    self.q0, 
                                                                                    self.f, 
                                                                                    self.actual_state,
                                                                                    self.empty_symbol) 
        return cadena

In [2]:
def fancy_printing(path):
    """
    Function to print the path in a more understandable way
    
    """
    
    for i in range(0,len(path),2):
            
        print("state: ", path[i])
        
        if i+1 < len(path):
            print("operation: ", path[i+1])
            print("")
        
                

# Implementation explanation

This time the automaton is made using backtracking, however, this time when it gets a valid path, it stops, so it does not try to find all the possible paths, that way the automaton is faster.

First, we are going only to explain the most importants methods.

### Empty transitions function

```

try:
    path = self.delta[(state, self.empty_symbol, stack_element)]
    element_to_stack = self.stack_operation[(state, self.empty_symbol, stack_element)]

    return path, element_to_stack

except:
    raise ErrorConfiguration("That transiction does not exist")
    
```

Comparing to the ndfa we did, now we only use recursivity in the process word method, so in this way we can easily print the path to process the word. In empty_transition path gets all the reacheable states given a state, and consuming an empty symbol, and consuming from the stack a given element of the stack or a empty symbol. In element_to_stack is to get the elements to be pushed to the stack given the same elements as in path.

### Process symbol

```

try:
    path = self.delta[(state, symbol, stack_element)]
    element_to_stack = self.stack_operation[(state, symbol, stack_element)]

    return path, element_to_stack

except:
    raise ErrorConfiguration("That transiction does not exist")
    
```

This function works as the last one, however, in this case we need a state, a symbol and stack elements, here empty symbols don't work.

### Process word

Here we are going to explain the parts of the code that cannot be understood by reading the comments

```

if (current_state, self.empty_symbol, self.stack[-1]) in self.delta:

    conditional = True

    path, element_to_stack = self.empty_transitions(current_state, self.stack[-1])

    memory = self.stack[-1]
    self.stack.pop(-1)

    for path_, element_ in zip(path, element_to_stack):
        paths.append("{}, {} --> {}".format(self.empty_symbol, memory, element_))
        paths.append(path_)
        if showing_backtrack:
            print(paths)
        if element_ != self.empty_symbol:
            self.stack.append(element_)


        stack_memory = self.stack[:]
        self._process_word_recursive(word, path_, paths, showing_backtrack)
        if self.trampa:
            return
        self.stack = stack_memory[:]
        paths.pop()
        paths.pop()

    self.stack = primal_stack_memory[:]
    
```

The function has a different *if* in which, the script to go through each one if possible, this if parts are almost equal the only differences are the elements, that recives, for example, one case is just processing empty symbol and using empty symbol of the stack, so each if is a specific case, the difference is that when the second element of the tuple within the if is empty_symbol in self._process_word_recursive it is passed word instead of word[1:], and another thing is that if the third element of the tuple is self.empty_symbol the last element of the stack is not eliminated, so if there is self.stack[-1] the last element of the stack is eliminated.


The for within the ifs is for going through each reacheable state along its element to push.

The stack_memory is to save the state of the stack before going to the recursive, so when it ends, it can go back to the original state before the recursivity, the primal stack memory is used for the stack to get to its original state before going to each if if self.trampa is used to cut the backtracking if a valid paths has been found

# Implementation Example 1

![image.png](images/pushdown.PNG)

In [3]:
example = pushdown_automata(empty_symbol = 'epsilon')

<ipython-input-1-fa3ac6a68de9>:57: UserWarning: Not valid automaton
  warnings.warn("Not valid automaton")


In [4]:
example.load_from_file("pushdown_examples/pushdown.txt")

### checking the stack element after an operation

In [5]:
example.stack_operation

{('q1', 'epsilon', 'epsilon'): ['$'],
 ('q2', '0', 'epsilon'): ['0'],
 ('q2', '1', '0'): ['epsilon'],
 ('q3', '1', '0'): ['epsilon'],
 ('q3', 'epsilon', '$'): ['epsilon']}

### Checking the operations

In [6]:
example.delta

{('q1', 'epsilon', 'epsilon'): ['q2'],
 ('q2', '0', 'epsilon'): ['q2'],
 ('q2', '1', '0'): ['q3'],
 ('q3', '1', '0'): ['q3'],
 ('q3', 'epsilon', '$'): ['q4']}

### Methods examples

In [7]:
example.empty_transitions('q1')

(['q2'], ['$'])

In [8]:
example.process_symbol('q2','0', 'epsilon')

(['q2'], ['0'])

In [9]:
x,y = example.process_word('000111', False)

In [10]:
fancy_printing(y)

state:  q1
operation:  epsilon, epsilon --> $

state:  q2
operation:  0, epsilon --> 0

state:  q2
operation:  0, epsilon --> 0

state:  q2
operation:  0, epsilon --> 0

state:  q2
operation:  1, 0 --> epsilon

state:  q3
operation:  1, 0 --> epsilon

state:  q3
operation:  1, 0 --> epsilon

state:  q3
operation:  epsilon, $ --> epsilon

state:  q4


In [11]:
str(example)

"Grammar: \n q: ['q1', 'q2', 'q3', 'q4'] \n sigma: ['0', '1'], sigma stack: ['0', '$'],delta: {('q1', 'epsilon', 'epsilon'): ['q2'], ('q2', '0', 'epsilon'): ['q2'], ('q2', '1', '0'): ['q3'], ('q3', '1', '0'): ['q3'], ('q3', 'epsilon', '$'): ['q4']}, q0: q1, f: ['q1', 'q4'], current_sate: ['q4'], empty_symbol: epsilon"

In [12]:
example.actual_state

['q4']

In [13]:
example.process_word('00111')

['q1', 'epsilon, epsilon --> $', 'q2']
['q1', 'epsilon, epsilon --> $', 'q2', '0, epsilon --> 0', 'q2']
['q1', 'epsilon, epsilon --> $', 'q2', '0, epsilon --> 0', 'q2', '0, epsilon --> 0', 'q2']
['q1', 'epsilon, epsilon --> $', 'q2', '0, epsilon --> 0', 'q2', '0, epsilon --> 0', 'q2', '1, 0 --> epsilon', 'q3']
['q1', 'epsilon, epsilon --> $', 'q2', '0, epsilon --> 0', 'q2', '0, epsilon --> 0', 'q2', '1, 0 --> epsilon', 'q3', '1, 0 --> epsilon', 'q3']
['q1', 'epsilon, epsilon --> $', 'q2', '0, epsilon --> 0', 'q2', '0, epsilon --> 0', 'q2', '1, 0 --> epsilon', 'q3', '1, 0 --> epsilon', 'q3', 'epsilon, $ --> epsilon', 'q4']


False

As you can see that is not a valid word, however, in the backtraking shows a posible way of how can be a correct word, by deleting one 1

In [14]:
example.process_word('000001111', False)

False

# Implementation example 2
![image.png](images/pushdown2.PNG)

In [15]:
example2 = pushdown_automata(empty_symbol = 'epsilon')

<ipython-input-1-fa3ac6a68de9>:57: UserWarning: Not valid automaton
  warnings.warn("Not valid automaton")


In [16]:
example2.load_from_file("pushdown_examples/pushdown2.txt")

#### case of an empty string

In [17]:
boolean, path = example2.process_word('')

['q1', 'epsilon, epsilon --> $', 'q2']
['q1', 'epsilon, epsilon --> $', 'q2', 'q2', 'epsilon, epsilon --> epsilon', 'q3']
['q1', 'epsilon, epsilon --> $', 'q2', 'q2', 'epsilon, epsilon --> epsilon', 'q3', 'q3', 'epsilon, $ --> epsilon', 'q4']


In [18]:
fancy_printing(path)

state:  q1
operation:  epsilon, epsilon --> $

state:  q2
operation:  q2

state:  epsilon, epsilon --> epsilon
operation:  q3

state:  q3
operation:  epsilon, $ --> epsilon

state:  q4
operation:  q4



In [19]:
boolean, path = example2.process_word('aabbc')

['q1', 'epsilon, epsilon --> $', 'q2']
['q1', 'epsilon, epsilon --> $', 'q2', 'a, epsilon --> a', 'q2']
['q1', 'epsilon, epsilon --> $', 'q2', 'a, epsilon --> a', 'q2', 'a, epsilon --> a', 'q2']
['q1', 'epsilon, epsilon --> $', 'q2', 'a, epsilon --> a', 'q2', 'a, epsilon --> a', 'q2', 'epsilon, epsilon --> epsilon', 'q3']
['q1', 'epsilon, epsilon --> $', 'q2', 'a, epsilon --> a', 'q2', 'a, epsilon --> a', 'q2', 'epsilon, epsilon --> epsilon', 'q3', 'b, a --> epsilon', 'q3']
['q1', 'epsilon, epsilon --> $', 'q2', 'a, epsilon --> a', 'q2', 'a, epsilon --> a', 'q2', 'epsilon, epsilon --> epsilon', 'q3', 'b, a --> epsilon', 'q3', 'b, a --> epsilon', 'q3']
['q1', 'epsilon, epsilon --> $', 'q2', 'a, epsilon --> a', 'q2', 'a, epsilon --> a', 'q2', 'epsilon, epsilon --> epsilon', 'q3', 'b, a --> epsilon', 'q3', 'b, a --> epsilon', 'q3', 'epsilon, $ --> epsilon', 'q4']
['q1', 'epsilon, epsilon --> $', 'q2', 'a, epsilon --> a', 'q2', 'a, epsilon --> a', 'q2', 'epsilon, epsilon --> epsilon', 'q3'

In [20]:
fancy_printing(path)

state:  q1
operation:  epsilon, epsilon --> $

state:  q2
operation:  a, epsilon --> a

state:  q2
operation:  a, epsilon --> a

state:  q2
operation:  epsilon, epsilon --> epsilon

state:  q3
operation:  b, a --> epsilon

state:  q3
operation:  b, a --> epsilon

state:  q3
operation:  epsilon, $ --> epsilon

state:  q4
operation:  c, epsilon --> epsilon

state:  q4


In [21]:
example2.actual_state

'q4'

In [22]:
example2.stack

[]

In [23]:
example2.process_word('abbc', False)

(True,
 ['q1',
  'epsilon, epsilon --> $',
  'q2',
  'a, epsilon --> a',
  'q2',
  'epsilon, epsilon --> epsilon',
  'q5',
  'b, epsilon --> epsilon',
  'q5',
  'b, epsilon --> epsilon',
  'q5',
  'epsilon, epsilon --> epsilon',
  'q6',
  'c, a --> epsilon',
  'q6',
  'epsilon, $ --> epsilon',
  'q7'])

In [24]:
example2.process_word('acbbc', False)

False

# Implementation example 3
![image.png](images/pushdown3.PNG)

In [25]:
example3 = pushdown_automata(empty_symbol = 'epsilon')
example3.load_from_file("pushdown_examples/pushdown3.txt")

<ipython-input-1-fa3ac6a68de9>:57: UserWarning: Not valid automaton
  warnings.warn("Not valid automaton")


In [26]:
example3.delta

{('q1', 'epsilon', 'epsilon'): ['q2'],
 ('q2', '0', 'epsilon'): ['q2'],
 ('q2', '1', 'epsilon'): ['q2'],
 ('q2', 'epsilon', 'epsilon'): ['q3'],
 ('q3', '0', '0'): ['q3'],
 ('q3', '1', '1'): ['q3'],
 ('q3', 'epsilon', '$'): ['q4']}

In [27]:
example3.stack_operation

{('q1', 'epsilon', 'epsilon'): ['$'],
 ('q2', '0', 'epsilon'): ['0'],
 ('q2', '1', 'epsilon'): ['1'],
 ('q2', 'epsilon', 'epsilon'): ['epsilon'],
 ('q3', '0', '0'): ['epsilon'],
 ('q3', '1', '1'): ['epsilon'],
 ('q3', 'epsilon', '$'): ['epsilon']}

In [28]:
example3.process_word('01', False)

False

In [33]:
example3.process_word('01100', False)

False

In [29]:
example3.process_word('0110', False)

(True,
 ['q1',
  'epsilon, epsilon --> $',
  'q2',
  '0, epsilon --> 0',
  'q2',
  '1, epsilon --> 1',
  'q2',
  'epsilon, epsilon --> epsilon',
  'q3',
  '1, 1 --> epsilon',
  'q3',
  '0, 0 --> epsilon',
  'q3',
  'epsilon, $ --> epsilon',
  'q4'])

In [30]:
boolean, path = example3.process_word('01011010', False)

In [31]:
fancy_printing(path)

state:  q1
operation:  epsilon, epsilon --> $

state:  q2
operation:  0, epsilon --> 0

state:  q2
operation:  1, epsilon --> 1

state:  q2
operation:  0, epsilon --> 0

state:  q2
operation:  1, epsilon --> 1

state:  q2
operation:  epsilon, epsilon --> epsilon

state:  q3
operation:  1, 1 --> epsilon

state:  q3
operation:  0, 0 --> epsilon

state:  q3
operation:  1, 1 --> epsilon

state:  q3
operation:  0, 0 --> epsilon

state:  q3
operation:  epsilon, $ --> epsilon

state:  q4


# Implementaion Example of invalid automaton

In [32]:
example4 = pushdown_automata(empty_symbol = 'epsilon')
example4.load_from_file("pushdown_examples/pushdown_error.txt")

<ipython-input-1-fa3ac6a68de9>:57: UserWarning: Not valid automaton
  warnings.warn("Not valid automaton")


TypeError: The stack element to push does not belong to the variables for the stack